In [4]:
#Reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
from google.colab import files
uploaded = files.upload()

Saving rentalleasingcompany_tweets.csv to rentalleasingcompany_tweets.csv


In [0]:
import pandas as pd


import io
df = pd.read_csv(io.BytesIO(uploaded['rentalleasingcompany_tweets.csv']),sep='|', names = ["tweets"])

In [0]:
# !pip install pyLDAvis

In [9]:
#Cleaning
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use','take','help','also','tonight','location','simple','lot','need','come','give','s','buy','want','twitter','launch','still','get','try','thank','always','well','full','look','start','time','set','close','good','see','know','year','think','find','be','great','say','look','hour','remember','number','wwwhuatsg','iloggo','find','self','case','even','probity'])

In [0]:
#delete all symbols
#Delete mentions, RTs
#Delete links

df.head()

#Remove mentions and RTs
df['tweets'] = df['tweets'].str.replace('\S*@\S*\s?',"",regex=True)
df['tweets'] = df['tweets'].str.replace('\S*#\S*\s?',"",regex=True)
df['tweets'] = df['tweets'].str.replace('RT',"")
df['tweets'] = df['tweets'].str.replace('&amp',"")
# Remove links
df['tweets'] = df['tweets'].str.replace('http\S+',"",regex=True)

# Only retain characters and spaces
df['tweets'] = df['tweets'].str.replace('[^a-zA-Z ]+',"",regex=True)

#Remove duplicates and blanks
df['tweets'] = df['tweets'].replace('', np.nan)
df.dropna(subset=['tweets'], inplace=True)
df.drop_duplicates(subset ="tweets", keep = False, inplace = True) 

In [0]:
df.tail()

,tweets
25142.0,No its not a Mini
25143.0,Congratulations on making the Most Influenti...
25144.0,Oh yes
25145.0,This weeks is the iconic Honda CRX
25146.0,Here is my hire car for the next week a Its ...


In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
from gensim.parsing.preprocessing import remove_stopwords
docs = list(df['tweets'])

docs2 = []
for sent in docs:
  docs2.append(remove_stopwords(sent))

docs = docs2


from nltk.corpus import stopwords
# from nltk.stem.wordnet import WordNetLemmatizer

from nltk.stem import PorterStemmer 
ps = PorterStemmer() 

import string

stop = set(stopwords.words('english')) 
stop_list = ['carlease','gave','leasing','lease','rent','rental','car','cars','vehicle','from', 'subject', 're', 'edu', 'use','take','help','also','tonight','location','simple','lot','need','come','give','s','buy','want','twitter','launch','still','get','try','thank','always','well','full','look','start','time','set','close','good','see','know','year','think','find','be','great','say','look','hour','remember','number','wwwhuatsg','iloggo','find','self','case','even','probity',
'like','find','good','weve','branch','star','uk','friday','know','u','take','right','let','month','youre','please','dm','number','send','dr','c','k','here','dont','pm','th','like','find','sure','hi']


exclude = set(string.punctuation)
# lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop and i not in stop_list])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(ps.stem(word) for word in punc_free.split())
    return normalized

docs = [clean(doc).split() for doc in docs] 

In [12]:
# from nltk.stem.wordnet import WordNetLemmatizer
# # nltk.download('wordnet')

from nltk.stem import PorterStemmer 
ps = PorterStemmer() 

# Lemmatize all words in documents.
# lemmatizer = WordNetLemmatizer()
docs = [[ps.stem(token) for token in doc ] for doc in docs]


def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
data_words_nostops = remove_stopwords(docs)

NameError: ignored

In [0]:
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [0]:
# Remove rare and common tokens.

from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20,no_above=0.3)

In [0]:
# Vectorize data.

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [15]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 1112
Number of documents: 18674


In [21]:
!wget https://github.com/rwalk/gsdmm/blob/master/gsdmm/mgp.py

# STTM lib from Github

#!pip install git+https://github.com/rwalk/gsdmm.git

from gsdmm import MovieGroupProcess

# mgp = MovieGroupProcess(K=25, alpha=0.1, beta=0.1, n_iters=30)
mgp = MovieGroupProcess(K=25, alpha=0.1, beta=0.1, n_iters=30)
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)

y = mgp.fit(docs, n_terms)

--2019-10-19 00:56:57--  https://github.com/rwalk/gsdmm/blob/master/gsdmm/mgp.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘mgp.py.3’

mgp.py.3                [  <=>               ] 116.65K   214KB/s    in 0.5s    

2019-10-19 00:56:59 (214 KB/s) - ‘mgp.py.3’ saved [119449]

In stage 0: transferred 16730 clusters with 25 clusters populated
In stage 1: transferred 11130 clusters with 25 clusters populated
In stage 2: transferred 7898 clusters with 25 clusters populated
In stage 3: transferred 6191 clusters with 25 clusters populated
In stage 4: transferred 5295 clusters with 25 clusters populated
In stage 5: transferred 4838 clusters with 25 clusters populated
In stage 6: transferred 4388 clusters with 25 clusters populated
In stage 7: transferred 4144 clusters with 25 clusters populated
In stage 8: transferred 3911 clust

In [0]:
from operator import itemgetter
from collections import OrderedDict

def top_words_cluster(clust_list,num_words):
  output_list = []
  for cluster in clust_list:
    sorted_dict = OrderedDict(sorted(cluster.items(), key=itemgetter(1)))
    #reverse and slice 
    sorted_dict = list(sorted_dict.items())[-num_words:]
    output_list.append(sorted_dict)
  return output_list

In [30]:
# from topic_allocation import top_words, topic_attribution

doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)
print('*'*20)
# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):', top_index)
print('*'*20)
# Show the top 5 words in term frequency for each cluster 

output_topics = top_words_cluster(mgp.cluster_word_distribution, 10)

pprint(output_topics[1])
print("\n")
pprint(output_topics[8])
print("\n")
pprint(output_topics[13])
print("\n")
pprint(output_topics[14])


Number of documents per topic : [ 435  848  149  452 2042  695 1093  288 2800 1216  492  628  290 2042
 1636   13  180   89  591  517   15  293   63 1296  511]
********************
Most important clusters (by number of docs inside): [ 8  4 13 14 23  9  6  1  5 11]
********************
[('rate', 81),
 ('check', 85),
 ('money', 95),
 ('today', 100),
 ('new', 103),
 ('fox', 104),
 ('day', 134),
 ('deal', 161),
 ('book', 174),
 ('save', 211)]


[('destin', 178),
 ('vacat', 212),
 ('guid', 214),
 ('plan', 215),
 ('best', 226),
 ('check', 255),
 ('trip', 271),
 ('famili', 305),
 ('tip', 375),
 ('travel', 850)]


[('seri', 97),
 ('hybrid', 100),
 ('concept', 106),
 ('rang', 128),
 ('bmw', 144),
 ('electr', 152),
 ('model', 166),
 ('suv', 204),
 ('reveal', 255),
 ('new', 826)]


[('check', 95),
 ('island', 95),
 ('drive', 101),
 ('beauti', 101),
 ('best', 108),
 ('hire', 139),
 ('explor', 148),
 ('road', 212),
 ('visit', 221),
 ('trip', 236)]
